In [1]:
import xarray as xr
import numpy as np
#import pandas as pd
import os
import glob
import copy as cp
import random
import pyproj as proj
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn import cluster
import wpca
import pandas as pd
import pickle 
from matplotlib.cm import get_cmap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.crs as ccrs
import cartopy as cr
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from datetime import datetime

# parameters

In [2]:
SUB_SETS = {'all':{'Subset_name':'/storage/nplanat/Glorys12_OP_journalier/NEW_RUN/Sub_samp_y_1993_-_2014_delta_1_100000_start_lon_0.nc',\
                  'init_year':1993,\
                  'final_year':2014,\
                   'delta_year':1,\
                  'N_particles':100000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/NEW_RUN/'},\
           'weird_path_only':{'Subset_name':'/storage/nplanat/Glorys12_OP_journalier/WEIRD/Sub_samp_y_1996_-_1997_-delta_1_40000_start_lon_0.nc',\
                  'init_year':1996,\
                  'final_year':1997,\
                   'delta_year':1,\
                  'N_particles':40000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/WEIRD/'},\
           'without_weird_path':{'Subset_name':None,\
                  'init_year':1993,\
                  'final_year':2014,\
                   'delta_year':1,\
                  'N_particles':100000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/NO_WEIRD/'},\
           'all_short':{'Subset_name':'/storage/nplanat/Glorys12_OP_journalier/SHORT/Sub_samp_y_1993_-_2014_delta_1_100000_start_lon_0.nc',\
                  'init_year':1993,\
                  'final_year':2014,\
                   'delta_year':1,\
                  'N_particles':100000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/SHORT/'}}

In [3]:
sel_subset = 'all_short'
load_previous_run = True
init_year = SUB_SETS[sel_subset]['init_year']
final_year = SUB_SETS[sel_subset]['final_year']
delta_year = SUB_SETS[sel_subset]['delta_year']
Subset_name = SUB_SETS[sel_subset]['Subset_name']
N_particles = SUB_SETS[sel_subset]['N_particles']
Subset_path = SUB_SETS[sel_subset]['Subset_path']

files = sorted(glob.glob('/storage/nplanat/Glorys12_OP_journalier/ADV_new_*'))
files_m = sorted(glob.glob('/storage/nplanat/Glorys12_OP_journalier/ADV_m_200_*'))
files_s = sorted(glob.glob('/storage/nplanat/Glorys12_OP_journalier/ADV_s_200_*'))

length_days = 450
path_save_prediction = Subset_path 


perctest = 0.1 
perctrain = 0.8
trans = True

#PCA 
n_components = 0.9999999

# Clustering
init = 'k-means++'
nmb_initialisations = 20  # number of initiatilisaton for the k-means++ 
max_iter = 500
tol = 5e-4
algorithm = 'full'
verbose = 0 
sample_weight = None
n_split = 20  # number of iterations for convergence
n_clusters = 20 #number of clusters

# Generate sub-set

In [4]:
def remove_short(ds): 
    l = [len(ds.sel(traj=i).lat.dropna(dim='obs')) for i in ds.traj]
    keep = np.array([i for i in range(len(l)) if l[i] > 90]) # more than 3 months
    ds2 = ds.sel(traj=xr.DataArray(keep)).rename_dims({'dim_0':'traj'}) 
    return ds2

In [12]:
def get_traj_sub(year_init, year_final, delta_year, files, length_days):
    dslist = []
    for yr in range(year_init,year_final+1,delta_year):
        print(yr)
        filesS = files[np.where(np.array([int(files[i][-7:-3]) for i in range(len(files))]) == yr)[0][0]]
        dsl = xr.open_dataset(filesS)
        dsl = remove_short(dsl) # remove short trajectories -- to avoid memory errors
        dsl = dsl.sel(obs = np.arange(length_days+366))
        dslist.append(dsl)

    
    return dslist

In [6]:
def random_sample(N_particles, ds):
    random.seed(4)
    sample = random.sample(list(ds.traj.values), k=N_particles) 
    ds_samp = ds.sel(traj=xr.DataArray(sample)).rename_dims({'dim_0':'traj'})  # select the random particles
    return ds_samp

In [19]:
try:
    ds_samp = xr.open_dataset(Subset_name)
except:
    dslist = get_traj_sub(init_year, final_year, delta_year, files, length_days)
    L_all = [x.compute() for x in dslist]
    ds = xr.concat(L_all, dim='traj')
    ds_samp = random_sample(N_particles, ds)
    ds_samp.to_netcdf(Subset_name)

1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


# Pre-processing

In [20]:
def split_sets(lats_all, lons_all, temps_all, sals_all, perctest, perctrain):
    
    s0, s1 = lats_all.shape
    data = np.concatenate((lats_all,lons_all), axis=1)
    
    random.seed(4)
    random.shuffle(data)
    random.seed(4)
    random.shuffle(temps_all)
    random.seed(4)
    random.shuffle(sals_all)

    lats_test = data[:int(perctest*s0), 0:s1]
    lons_test = data[:int(perctest*s0), s1:]

    lats_train = data[int(perctest*s0):int(perctest*s0)+int(perctrain*s0), 0:s1]
    lons_train = data[int(perctest*s0):int(perctest*s0)+int(perctrain*s0), s1:]

    lats_valid = data[int(perctest*s0)+int(perctrain*s0):, 0:s1]
    lons_valid = data[int(perctest*s0)+int(perctrain*s0):, s1:]

    temps_test = temps_all[:int(perctest*s0),:]
    sals_test = sals_all[:int(perctest*s0),:]
    
    temps_train =temps_all[int(perctest*s0):int(perctest*s0)+int(perctrain*s0),:]
    sals_train = sals_all[int(perctest*s0):int(perctest*s0)+int(perctrain*s0),:]
    
    temps_valid = temps_all[int(perctest*s0)+int(perctrain*s0):,:]
    sals_valid = sals_all[int(perctest*s0)+int(perctrain*s0):,:]
    
    return lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid, temps_train, temps_test, temps_valid, sals_train, sals_test, sals_valid

In [21]:
def lons_in_interval(start_lon, lons):
    #this should shift all longitudes so it lies in start_lon + 360. 
    if start_lon<0 : 
        print('start_lons should be in [0-360[')
        return None
    else:
        lons_shifted = lons
        while np.any(lons_shifted<0):
            lons_shifted[lons_shifted<0] = lons_shifted[lons_shifted<0]+360
        lons_2 = cp.copy(lons_shifted)
        lons_2[lons_shifted>=start_lon] = lons_shifted[lons_shifted>=start_lon] - start_lon
        lons_2[lons_shifted<start_lon] = lons_shifted[lons_shifted<start_lon] - start_lon +360
        return cp.copy(lons_2)

In [22]:
def extract(path, length_days) :
    ds = xr.open_dataset(path)
    ds = remove_short(ds)
    lats = ds.lat[:,0:length_days].values
    lons_i = ds.lon[:,0:length_days].values
    lons = lons_in_interval(0, lons_i) # by default start_lon = 360
    temps = ds.temperature[:,0:length_days].values
    sal = ds.salinity[:,0:length_days].values
    z = ds.z[:,0:length_days].values
    date = ds.time[:,0].values
    return lats, lons, temps, sal, date, z

In [23]:
def translate(lat, lon):
    return lat-np.repeat(lat[:,0][:, np.newaxis], lat.shape[1], axis=1), lon-np.repeat(lon[:,0][:, np.newaxis], lon.shape[1], axis=1)

In [24]:
def Part_1_pre_processing_one_sort(t, la, lo):
    if t == True :
        lats_train_pp, lons_train_pp = translate(la, lo)

    # Reshape
    X_lats_lons_train = np.concatenate((lats_train_pp, lons_train_pp), axis = 1) #concatenation of features to make it in the correct shape
    X_lats_lons_train[np.isnan(X_lats_lons_train)] = 0 
   
    return X_lats_lons_train

In [25]:
def Part_1_pre_processing(Dataset_path, length_days, perctest, perctrain, t):
    lats_all,lons_all, temps_all, sals_all, date_all, depth_all = extract(Dataset_path, length_days)
    lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid, temps_train, temps_test, temps_valid, sals_train, sals_test, sals_valid = split_sets(lats_all, lons_all,temps_all, sals_all, perctest, perctrain)

    print('Pre_processing')
    # PRE-PROCESSING
    X_lats_lons_train = Part_1_pre_processing_one_sort(t, lats_train, lons_train)
    X_lats_lons_test = Part_1_pre_processing_one_sort(t, lats_test, lons_test)
    X_lats_lons_valid = Part_1_pre_processing_one_sort(t, lats_valid, lons_valid)
    
    return X_lats_lons_valid, X_lats_lons_test, X_lats_lons_train, lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid,

In [26]:
X_lats_lons_valid, X_lats_lons_test, X_lats_lons_train, lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid = Part_1_pre_processing(Subset_name, length_days, perctest, perctrain, trans)

Pre_processing


# PCA

In [27]:
pca = PCA(n_components=n_components)#, copy=copy, whiten=whiten, svd_solver = svd_solver)

In [28]:
X_reduced_train = pca.fit_transform(X_lats_lons_train)
N_features_PCA = pca.n_features_
N_samples_PCA = pca.n_samples_
N_components_PCA = pca.n_components_
Explained_variance_ratio_PCA = pca.explained_variance_ratio_
print('The number of initial features was: ', N_features_PCA)
print('The number of selected features is: ', N_components_PCA)
print('The number of samples is: ', N_samples_PCA)
print('The explained variance desired is:', n_components, '%, the obtained variance explained is: ', np.sum(Explained_variance_ratio_PCA), '%')

The number of initial features was:  900
The number of selected features is:  752
The number of samples is:  80000
The explained variance desired is: 0.9999999 %, the obtained variance explained is:  0.9999999017605582 %


In [29]:
X_reduced_valid = pca.transform(X_lats_lons_valid)
X_reduced_test  = pca.transform(X_lats_lons_test)

In [30]:
#save PCA : 
with open(Subset_path + 'pca.pkl', 'wb') as pickle_file:
        pickle.dump(pca, pickle_file)

# Clustering

In [31]:
def kmeans(n_splits, X_reduced, n_clusters, init, nmb_initialisations, max_iter, tol, algorithm, verbose, sample_weight):

    kf = KFold(n_splits=n_splits, shuffle=True)
    avg_silouhette = 0
    i = 0
    for train_index, val_index in kf.split(X_reduced):
        print('fold nmb ', i)
        i+=1
        X_train = X_reduced[train_index,:]
        k_means = cluster.KMeans(n_clusters=n_clusters, init=init, n_init = nmb_initialisations, max_iter = max_iter, tol = tol, algorithm = algorithm, verbose = verbose)
        k_means.fit(X_train, sample_weight = sample_weight)
        X_centers = k_means.cluster_centers_
        a_temp = k_means.inertia_
        if a_temp>avg_silouhette:
            X_centered_memory = X_centers
            k_means_memory = k_means
            avg_silouhette = a_temp
    
    return X_centered_memory, k_means_memory, a_temp

In [32]:
Traj_centroids, kmeans, score = kmeans(n_split, X_reduced_train, n_clusters, init, nmb_initialisations, max_iter, tol, algorithm, verbose, sample_weight)

fold nmb  0
fold nmb  1
fold nmb  2
fold nmb  3
fold nmb  4
fold nmb  5
fold nmb  6
fold nmb  7
fold nmb  8
fold nmb  9
fold nmb  10
fold nmb  11
fold nmb  12
fold nmb  13
fold nmb  14
fold nmb  15
fold nmb  16
fold nmb  17
fold nmb  18
fold nmb  19


In [33]:
# clustering
np.save(Subset_path + 'Traj_centroids', Traj_centroids)

# Prediction

In [34]:
def predict(X_reduced_val, X_centers):
    
    X_reduced_val2 = np.repeat(X_reduced_val[:,:,np.newaxis], X_centers.shape[0], axis = 2)
    X_centers2 = np.repeat(np.transpose(X_centers)[np.newaxis,:,:], X_reduced_val.shape[0], axis = 0)
    
    return np.argmin(np.linalg.norm(X_reduced_val2-X_centers2, axis=1), axis = 1)

In [35]:
Labels_valid = predict(X_reduced_valid, Traj_centroids)
Labels_test = predict(X_reduced_test, Traj_centroids)

In [36]:
np.save(path_save_prediction+'Labels_valid', Labels_valid)
np.save(path_save_prediction+'Labels_test', Labels_test)
np.save(path_save_prediction+'lats_test', lats_test)
np.save(path_save_prediction+'lons_test', lons_test)
np.save(path_save_prediction+'lats_train', lats_train)
np.save(path_save_prediction+'lons_train', lons_train)
np.save(path_save_prediction+'lats_valid', lats_valid)
np.save(path_save_prediction+'lons_valid', lons_valid)

# Generate Lats-Lons  all

In [37]:
def extract_dates(path, length_days) :
    ds = xr.open_dataset(path)
    ds = remove_short(ds)
    date = ds.time[:,0].values
    return date
def extract_all_dates_year(yr, length_days, all_filles) :
    file = all_filles[np.where(np.array([int(all_filles[i][-7:-3]) for i in range(len(all_filles))]) == yr)[0][0]]
    date = extract_dates(file, length_days)    
    return date
def extract_all_per_year(yr, length_days, all_filles) :
    file = all_filles[np.where(np.array([int(all_filles[i][-7:-3]) for i in range(len(all_filles))]) == yr)[0][0]]
    lats, lons, temps, sal, date, depth = extract(file, length_days)    
    return lats, lons, temps, sal, date, depth

In [38]:
try : 
    np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/times_2013.npy', allow_pickle = True)
except:
    for yr in range(1993,2014):
        print('Year', yr)
        lats_all, lons_all, temps_all, sals_all, date_all, prof_all = extract_all_per_year(yr, length_days, files)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/lats_%i'%yr, lats_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/depths_%i'%yr, prof_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/temps_%i'%yr, temps_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/sals_%i'%yr, sals_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/lons_%i'%yr, lons_all)
        date_all = extract_all_dates_year(yr, length_days, files)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/times_%i'%yr, date_all)

In [39]:
try : 
    np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_times_2013.npy', allow_pickle = True)
except:
    for yr in range(1993,2014):
        print('Year', yr)
        lats_all, lons_all, temps_all, sals_all, date_all, prof_all = extract_all_per_year(yr, length_days, files_m)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_lats_%i'%yr, lats_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_depths_%i'%yr, prof_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_temps_%i'%yr, temps_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_sals_%i'%yr, sals_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_lons_%i'%yr, lons_all)
        date_all = extract_all_dates_year(yr, length_days, files_m)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_times_%i'%yr, date_all)

In [40]:
try : 
    np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_times_2013.npy', allow_pickle = True)
except:
    for yr in range(1993,2014):
        print('Year', yr)
        lats_all, lons_all, temps_all, sals_all, date_all, prof_all = extract_all_per_year(yr, length_days, files_s)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_lats_%i'%yr, lats_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_depths_%i'%yr, prof_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_temps_%i'%yr, temps_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_sals_%i'%yr, sals_all)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_lons_%i'%yr, lons_all)
        date_all = extract_all_dates_year(yr, length_days, files_s)
        np.save('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_times_%i'%yr, date_all)